## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("c://users/sophc/onedrive/desktop/classwork/world_weather_analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.1927,168.8643,50.49,87,100,5.12,light rain
1,1,Kloulklubed,PW,7.0419,134.2556,77.50,72,100,4.32,overcast clouds
2,2,Healesville,AU,-37.6500,145.5167,53.02,74,100,1.99,overcast clouds
3,3,Busselton,AU,-33.6500,115.3333,41.09,86,5,7.56,clear sky
4,4,Thompson,CA,55.7435,-97.8558,51.96,62,90,17.27,overcast clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 75
What is the maximun temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kloulklubed,PW,7.0419,134.2556,77.50,72,100,4.32,overcast clouds
5,5,Puerto Ayora,EC,-0.7393,-90.3518,76.96,86,99,5.99,overcast clouds
18,18,Camacha,PT,33.0833,-16.3333,76.53,73,20,8.05,few clouds
22,22,Sur,OM,22.5667,59.5289,87.55,61,59,6.98,broken clouds
31,31,Lata,PT,40.1629,-8.3327,77.04,77,66,2.95,broken clouds
32,32,Katsuura,JP,35.1333,140.3000,75.31,91,100,8.41,overcast clouds
33,33,Maluso,PH,6.5430,121.8753,78.89,85,100,5.53,overcast clouds
35,35,Jamestown,US,42.0970,-79.2353,75.02,50,40,13.00,scattered clouds
38,38,Cabo San Lucas,MX,22.8909,-109.9124,89.51,70,1,8.05,clear sky
41,41,Kapaa,US,22.0752,-159.3190,79.12,59,100,5.01,overcast clouds


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                243
City                   243
Country                242
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kloulklubed,PW,77.50,overcast clouds,7.0419,134.2556,
5,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,
18,Camacha,PT,76.53,few clouds,33.0833,-16.3333,
22,Sur,OM,87.55,broken clouds,22.5667,59.5289,
31,Lata,PT,77.04,broken clouds,40.1629,-8.3327,
32,Katsuura,JP,75.31,overcast clouds,35.1333,140.3000,
33,Maluso,PH,78.89,overcast clouds,6.5430,121.8753,
35,Jamestown,US,75.02,scattered clouds,42.0970,-79.2353,
38,Cabo San Lucas,MX,89.51,clear sky,22.8909,-109.9124,
41,Kapaa,US,79.12,overcast clouds,22.0752,-159.3190,


In [14]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kloulklubed,PW,77.50,overcast clouds,7.0419,134.2556,Storyboard Beach Resort
5,Puerto Ayora,EC,76.96,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
18,Camacha,PT,76.53,few clouds,33.0833,-16.3333,Hotel Porto Santo & Spa
22,Sur,OM,87.55,broken clouds,22.5667,59.5289,Sur Plaza Hotel
31,Lata,PT,77.04,broken clouds,40.1629,-8.3327,Casa Maquia
32,Katsuura,JP,75.31,overcast clouds,35.1333,140.3000,Katsuura Hotel Mikazuki
33,Maluso,PH,78.89,overcast clouds,6.5430,121.8753,RUSTANTO Fam
35,Jamestown,US,75.02,scattered clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown
38,Cabo San Lucas,MX,89.51,clear sky,22.8909,-109.9124,Hotel Tesoro Los Cabos
41,Kapaa,US,79.12,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort


In [18]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [21]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [25]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))